# Regras de associação: Groceries

##1 - Instalação das bibliotecas



In [3]:
import signal
import os
from collections import Counter
from itertools import combinations
from datetime import datetime


import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


## Definição do algoritmo apriori

In [4]:
def apriori_algoritmo(np_base_binaria, min_sup = 0.2, timeout = 120):

    def timeout_function(signum, frame):
        raise Exception("Timeout exceeded. Choose a higher min_sup or let algorithm work more by increasing timeout")

    num_itens = len(np_base_binaria[0])
    num_transacoes = len(np_base_binaria)

    support_itemset = {}
    itens = range(num_itens)

    signal.signal(signal.SIGALRM, timeout_function)
    signal.alarm(timeout)

    for num_itemset in range(1, num_itens + 1):
        itemsets = combinations(itens, num_itemset)
        itens = set()
        for itemset in itemsets:
            frequency_itemset = 0
            for transacao in np_base_binaria:
                frequency_itemset += all(transacao[list(itemset)])
            if (frequency_itemset / num_transacoes) >= min_sup:
                itens.update(itemset)
                support_itemset[itemset] = frequency_itemset / num_transacoes
        if not itens:
            break

    return support_itemset

## 2- Obter os dados

## 3- Inspeção dos dados

In [5]:
dados = pd.read_csv('Groceries_dataset.csv')

In [8]:
dados.dtypes

Member_number       int64
Date               object
itemDescription    object
dtype: object

In [9]:
dados.head()

,Member_number,Date,itemDescription
0,1808,21-07-2015,tropical fruit
1,2552,05-01-2015,whole milk
2,2300,19-09-2015,pip fruit
3,1187,12-12-2015,other vegetables
4,3037,01-02-2015,whole milk


In [6]:
dados.count()

Member_number      38765
Date               38765
itemDescription    38765
dtype: int64

In [ ]:
dados.isna().sum(axis = 0)

Member_number      0
Date               0
itemDescription    0
dtype: int64

*Podemos analisar que o banco de dados não possui nenhum valor em branco*

Cada transação pode ser definida como a chave composta pelas colunas **Member_number** e **date**, ou seja, diz o cliente que fez a compra e a data da compra.

In [14]:
dados.itemDescription.nunique()

167

In [9]:
dados.groupby(['Member_number', 'Date']).ngroups

14963

O número de transações no conjunto de dados é 14963.

In [8]:
lista_ordenada_items = list(set(list(dados['itemDescription'])))

lista_ordenada_items.sort()

dados['codigoItem'] = dados['itemDescription'].apply(lambda item: lista_ordenada_items.index(item))

In [9]:
lista_ordenada_transacoes = list(set(list(zip(list(dados['Date']), list(dados['Member_number'])))))

lista_ordenada_transacoes.sort()

dados['codigoTransacao'] = np.vectorize(lambda x, y : lista_ordenada_transacoes.index((x,y)))(dados['Date'], dados['Member_number'])

In [10]:
items = dados[['codigoItem', 'itemDescription']].drop_duplicates().sort_values(by='codigoItem')

In [11]:
items.head()

,codigoItem,itemDescription
457,0,Instant food products
295,1,UHT-milk
3972,2,abrasive cleaner
980,3,artif. sweetener
1489,4,baby cosmetics


In [12]:
transacoes = dados[['codigoTransacao','Member_number', 'Date']].drop_duplicates().sort_values(by='codigoTransacao')

In [13]:
transacoes.head()

,codigoTransacao,Member_number,Date
10065,0,1249,01-01-2014
15338,1,1381,01-01-2014
11134,2,1440,01-01-2014
15535,3,1659,01-01-2014
12744,4,1789,01-01-2014


In [15]:
transacoesvsprodutos = dados[['codigoTransacao', 'codigoItem']]

In [16]:
transacoesvsprodutos.head(10)

,codigoTransacao,codigoItem
0,10064,156
1,2031,164
2,9149,109
3,5844,102
4,74,164
5,6470,122
6,3660,102
7,11271,112
8,9403,164
9,5465,156


In [17]:
transacoes_crosstab = pd.crosstab(transacoesvsprodutos['codigoTransacao'], transacoesvsprodutos['codigoItem'])

In [18]:
apriori_results = apriori_algoritmo(transacoes_crosstab.to_numpy(), min_sup=0.004, timeout = 240)

In [111]:
itens_dict = {d['codigoItem'] : d['itemDescription'] for d in items.to_dict('records')}

In [131]:
apriori_results_traduzido = {tuple(itens_dict[codigoItem] for codigoItem in result) : apriori_results[result] for result in apriori_results}

In [138]:
dados['itemDescription'].drop_duplicates().to_csv('itemDescription')

In [157]:
apriori_results_traduzido[('newspapers','whole milk')] / apriori_results_traduzido[('whole milk',)]

0.03554803216250529

In [158]:
apriori_results_traduzido[('newspapers', 'whole milk')] / (apriori_results_traduzido[('whole milk',)] * apriori_results_traduzido[('newspapers',)])

0.9139264694975372